# Hidden Markov Model
关于HMM的示例，可以参考[代码](../code/5_hmm.ipynb),如果需要实现可用性强的模型，则可以使用hmmlearn工具包

## Introduction

### HMM的定义

> 隐马尔科夫模型是关于时序的概率模型，描述由一个隐藏的马尔科夫链随机生成不可观测的状态随机序列，再由各个状态生成一个观测从而产生观测随机序列的过程。隐藏的马尔科夫链随机生成的状态的序列，称为状态序列；每个状态生成一个观测，而由此产生的观测的随机序列，称为观测序列。序列的每一个为止又可以看做是一个时刻。

马尔科夫模型由初始化概率分布、状态转移概率分布以及观测概率分布确定。

假设$Q$是所有可能的状态集合，$V$是所有可能的观测集合
$$
Q = \{q_1, q_2, \ldots, q_N\}, V = \{v_1, v_2, \ldots, v_M\}
$$
    A是状态转移概率矩阵:
$$
A = [a_{ij}]_{N\times N}\tag{1}
$$

$$
a_{ij} = P(i_{t+1} = q_j|i_t = q_i),\ \ \ i = 1, 2, \ldots, N;\ j = 1, 2,\ldots, N \tag{2}
$$
是在时刻t处于状态$q_i$的条件下在时刻t+1转移到状态$q_j$的概率。


B是观测概率矩阵
$$
 B = [b_j(k)]_{N\times M}\tag{3}
$$

$$
b_j(k) = P(o_t = v_k|i_t = q_j), \ \ \ j = 1, 2, \ldots, N; \ k = 1, 2, \ldots, M\tag{4}
$$

是在时刻t处于状态$q_j$条件下生成观测$v_k$的概率。


$\pi$是初始状态概率向量
$$
\pi= (\pi_i)\tag{5}
$$

$$
\pi_i = P(i_1 = q_i), \ \ \ i = 1, 2, \ldots, N\tag{6}
$$
是时刻t = 1处于状态$q_i$的概率。

因此，隐马尔可夫模型$\lambda$可以用三元符号表示，即
$$
\lambda = (A, B, \pi) \tag{7}
$$
**注意，HMM的假设**

>1) 每一时刻的隐藏状态仅与前一刻的状态有关
>
>2) 每一个时刻的观测状态仅与当前隐藏状态有关

### 观测序列的生成

根据HMM的定义，可以将一个长度为T的观测序列$O = (o_1, o_2, \ldots, o_T)$的生成过程描述如下：

> Input: HMM的$\lambda = (A, B, \pi)$，观测序列长度$T$
>
> Output: 观测序列$O = (o_1, o_2, \ldots, o_T)$
>
> (1) 按照初始状态分布$\pi$产生状态$i_1$
>
> (2) 令t = 1
>
> (3) 按照状态$i_t$的观测概率分布$b_j(k)$生成$o_t$
>
> (4) 按照状态$i_t$的状态转移概率分布$\{a_{i_ti_{t+1}}\}$产生状态$i_{t + 1}$, $i_{t + 1} = 1, 2, \ldots, N;$
>
> (5) 令$t = t + 1$，如果$t \lt T$，转到(3)，否则，停止

### HMM的三个基本问题

隐马尔可夫模型由三个基本问题

1. 概率计算问题。给定模型$\lambda = (A, B, \pi)$和观测序列$O = (o_1, o_2, \ldots, o_T)$,计算在模型$\lambda$下观测序列$O$出现的概率$P(O|\lambda)$
2. 学习问题。已知观测序列$O = (o_1, o_2, \ldots, o_T)$,估计模型$\lambda = (A, B, \pi)$参数，使得在该模型下观测序列概率$P(O|\lambda)$最大，即用极大似然估计的方法更新参数(一种[EM](em.ipynb)变体)
3. 预测问题, 也称为解码问题，给定模型$\lambda = (A, B, \pi)$和观测序列$O = (o_1, o_2, \ldots, o_T)$,求对给定观测序列条件概率$P(I|O)$最大的状态序列$I = (i_1, i_2, \ldots, i_T)$。即给定观测序列，求最有可能的对应的状态序列。

## Probability Computing

### 直接计算法

直接计算的方法就是列举所有可能长度T的状态序列，然后计算其生成对应观测序列的概率。所以可能求和，公式表示如下：
$$
P(I|\lambda) = \pi_{i_1}a_{i_1i_2}a_{i_2i_3}\ldots a_{i_{T-1}i_T}
\tag{8}
$$

$$
P(O|I, \lambda) = b_{i_1}(o_1)b_{i_2}(o_2)\ldots b_{i_T}(o_T)\tag{9}
$$

$$
\begin{aligned}
P(O|\lambda) &= \displaystyle\sum_I P(O|I, \lambda)P(I|\lambda)\\
& =\sum_{i_1, i_2, \ldots, i_T} \pi_{i_1}b_{i_1}(o_1)a_{i_1i_2}b_{i_2}(o_2)a_{i_2i_3}b_{i_3}(o_3)\ldots a_{i_{T-1}i_T}b_{i_T}(o_T)
\end{aligned}
\tag{10}
$$

当时利用公式(10)的计算方式，整个算法的时间复杂度为$O(TN^T)$

### 前向算法

**前向概率**: 给定隐马尔科夫模型$\lambda$，定义到时刻$t$部分观测序列为$o_1, o_2, \ldots, o_t$且当前状态为$q_i$的概率为前向概率，记作:
$$
\alpha_t(i) = P(o_1, o_2, \ldots, o_t, i_t = q_i|\lambda)\tag{11}
$$

算法过程如下:

> Input: 模型参数$\lambda$, 观测序列$O$
>
> Output: 观测序列概率$P(O|\lambda)$
>
> (1)初值

算法过程如下:

> Input: 模型参数$\lambda$, 观测序列$O$
>
> Output: 观测序列概率$P(O|\lambda)$
>
> (1)初值
> $$
\alpha_1(i) = \pi_ib_i(o_1), \ \ \ i = 1, 2, \ldots, N \tag{12}
$$
> 递推， 对$t = 1, 2, \ldots, T-1$
>$$
\alpha_{t +1}(i) = \bigg[\displaystyle\sum_{i =1}^N\alpha_t(j)a_{ji}\bigg]b_i(o_{t+1})\ \ \ i = 1, 2, \ldots, N\tag{13}
$$
> 停止
> $$
P(O|\lambda) = \displaystyle\sum_{i = 1}^N\alpha_T(i)\tag{14}
$$

### 后向算法

**后向概率**: 给定隐马尔科夫模型$\lambda$，定义从时刻$t+1$到$T$部分观测序列为$o_{t+1}, o_{t+2}, \ldots, o_T$且时刻$t$状态为$q_i$的概率为后向概率，记作:
$$
\beta_t(i) = P(o_{t+1}, o_{t+2}, \ldots, o_T, i_t = q_i|\lambda)\tag{15}
$$

算法过程如下:

> Input: 模型参数$\lambda$, 观测序列$O$
>
> Output: 观测序列概率$P(O|\lambda)$
>
> (1)初值,无论时刻$T$的状态是什么，到终止状态的概率均为1
> $$
\beta_T(i) = 1, \ \ \ i = 1, 2, \ldots, N \tag{16}
$$
> 递推， 对$t = T-1, T-2, \ldots, 1$
> $$
\beta_t(i) = \displaystyle\sum_{i =1}^Na_{ij}b_j(o_{t+1})\beta_{t+1}(j)\ \ \ i = 1, 2, \ldots, N\tag{17}
$$
> 停止
> $$
P(O|\lambda) = \displaystyle\sum_{i = 1}^N\pi_ib_i(o_1)\beta_1(i)\tag{18}
$$

### 一些概率值和期望的计算

1. 给定模型$\lambda$和观测$O$,在时刻$t$处于状态$q_i$的概率，记为
   $$
   \gamma_t(i) = P(i_t = q_i|O, \lambda)\tag{19}
   $$
	根据前向概率和后向概率的定义
	$$
	\begin{aligned}
	\gamma_t(i) &= P(i_t = q_i|O, \lambda)\\
	&= \frac{P(i_t = q_i, O| \lambda)}{P(O|\lambda)}\\
	&= \frac{\alpha_t(i)\beta_t(i)}{P(O|\lambda)}
	\end{aligned}\tag{20}
	$$

2. 给定模型$\lambda$和观测$O$,在时刻$t$处于状态$q_i$的概率，且在时刻$t+1$处于状态$q_j$的概率，记为
   $$
   \begin{aligned}
   \xi_t(i,j) &= P(i_t = q_i, i_{t+1} = q_j|O, \lambda)\\
   &= \frac{P(i_t = q_i, i_{t+1} = q_j, O| \lambda)}{P(O|\lambda)}\\
   &= \frac{\alpha_t(i)a_{ij}b_i(o_{t+1})\beta_{t+1}(j)}{P(O|\lambda)}
   \end{aligned}
   \tag{21}
   $$

## Parameter Estimation

### 监督学习

假设已给出的训练数据包含S个长度相同的观测序列和对应的状态序列$\{(O_1, I_1), (O_2, I_2), \ldots, (O_S, I_S)\}$，那么可以利用MLE来估计HMM的参数，具体方法如下。

1. 转移概率$a_{ij}$的估计

   假设样本中当前时刻状态为$q_i$,下一时刻的状态为$q_j$的频率为$A_{ij}$，那么
   $$
   \hat{a}_{ij} = \frac{A_{ij}}{\displaystyle\sum_{k= 1}^NA_{ik}}, \ \ \ i = 1, 2, \ldots, N;\ \ j = 1, 2, \ldots, N;\tag{22}
   $$

2. 观测概率$b_j(k)$的估计

   设样本中状态为$j$并观测为$k$的频数是$B_{jk}$,那么
   $$
   \hat{b}_j(k) = \frac{B_{jk}}{\displaystyle\sum_{k = 1}^M B_{jk}}, \ \ \ j = 1,2 ,\ldots, N;\ k = 1, 2, \ldots, M \tag{23}
   $$

3. 初始状态概率$\pi_i$的估计

   $\hat{\pi_i}$为S个样本中，初始状态为$q_i$的频率

### Baum-Welch Algorithm

假设给定训练数据只包含S个长度为T的观测序列${O_1, O_2, \ldots, O_S}$而没有对应的隐状态序列，目标是学习HMM的参数$\lambda = (A, B, \pi)$。解决方案是一种[EM](em.ipynb)算法。

1. 构造E步目标函数
   $$
   \begin{aligned}
   Q(\lambda, \bar{\lambda}) &=\arg \underset{\lambda}{\max} E_I\big[\log P(O,I|\lambda)\big|O, \bar{\lambda}\big] \\
   &=\arg \underset{\lambda}{\max} \displaystyle\sum_I\log P(O, I|\lambda)P(I|O, \bar{\lambda})\\
   &=\arg \underset{\lambda}{\max} \displaystyle\sum_I\log P(O, I|\lambda)\frac{P(I, O| \bar{\lambda})}{P(O|\bar{\lambda})}\\
   &= \arg \underset{\lambda}{\max} \displaystyle\sum_I\log P(O, I|\lambda)P(I, O| \bar{\lambda})\\
   \end{aligned}\tag{24}
   $$
   根据公式(8)、(9)，公式(24)改写为如下形式
   $$
   \begin{aligned}
   Q(\lambda, \bar{\lambda})
   = &\displaystyle\sum_I\log \pi_{i_1}P(I, O| \bar{\lambda})
   + \sum_I \bigg(\sum^{T-1}_{t = 1}\log a_{i_ti_{t+1}}\bigg)
   P(I, O| \bar{\lambda}) +\\
   & \sum_I\bigg(\sum^T_{t = 1}\log b_{i_t}(o_t)\bigg) P(I, O| \bar{\lambda})
   \end{aligned}\tag{25}
   $$
   进一步对公式变形
   $$
   \begin{aligned}
   Q(\lambda, \bar{\lambda})= 
   &\displaystyle\sum_{i = 1}^N\log \pi_{i_1}P(O, i_1 = i| \bar{\lambda})+\\
   & \sum_{i = 1}^N\sum_{j = 1}^N \sum^{T-1}_{t = 1}\log a_{ij}
   P(O, i_t = i, i_{t+1} = j| \bar{\lambda}) +\\
   & \sum_{j = 1}^N\sum^T_{t = 1}\log b_j(o_t)P(O, i_t = j| \bar{\lambda})
   \end{aligned}\tag{26}
   $$

2. 极大化目标函数， 最优化过程

   写出优化目标函数及其约束
   $$
   \begin{aligned}\max\ &\displaystyle\sum_{i = 1}^N\log \pi_{i_1}P(O, i_1 = i| \bar{\lambda})+\\& \sum_{i = 1}^N\sum_{j = 1}^N \sum^{T-1}_{t = 1}\log a_{ij}P(O, i_t = i, i_{t+1} = j| \bar{\lambda}) +\\& \sum_{j = 1}^N\sum^T_{t = 1}\log b_j(o_t)P(O, i_t = j| \bar{\lambda})\\s.t.\ &\sum_{i = 1}^N\pi_i = 1\\&\sum_{j = 1}^Na_{ij} = 1, \ \ i = 1,2, \ldots, N\\&\sum_{k = 1}^Mb_j(k) = 1, \ \ j = 1,2, \ldots, N\end{aligned}\tag{27}
   $$

   使用拉格朗日乘子法求解上述问题
   $$
   \pi_i = \frac{P(O, i_1 = i|\bar{\lambda})}{P(O|\lambda)}\tag{28}
   $$

   $$
   a_{ij} = \frac{\displaystyle\sum_{t = 1}^{T-1}P(O, i_t = i, i_{t+1} = j| \bar{\lambda})}{\displaystyle\sum_{t = 1}^{T-1}P(O, i_t = i|\lambda)}\tag{29}
   $$

   $$
   b_j(k) = \frac{\displaystyle\sum_{t = 1}^TP(O, i_t=j|\bar{\lambda})I(o_t = v_k)}{\displaystyle\sum_{t = 1}^TP(O, i_t = j|\bar{\lambda})}\tag{30}
   $$

   结合公式(20)、(21)，上述三式可以表示成
   $$
   \begin{aligned}\displaystyle&\pi_i = \gamma_1(i)\\&a_{ij} = \frac{\displaystyle\sum_{t = 1}^{T-1}\xi_t(i,j)}{\displaystyle\sum_{t = 1}^{T-1}\gamma_t(i)}\\&b_j(k) = \frac{\displaystyle\sum_{t = 1, o_t = v_k}^T\gamma_t(j)}{\displaystyle\sum_{t = 1}^T\gamma_t(j)}\end{aligned}
   $$

## Predict Algorithms

### 近似算法

近似算法的想法是在每个时刻$t$选择在该时刻最有可能出现的状态$i_t^*$，从而得到一个最优的状态序列作为预测结果。

给定HMM的参数$\lambda$和观测序列$O$，在时刻$t$处于状态$q_i$的概率$\gamma_t(i)$是
$$
\gamma_t(i) = \frac{\alpha_t(i)\beta_t(i)}{P(O|\lambda)}\tag{31}
$$
在每一个时刻最优可能的状态$i_t^*$是
$$
i_t^* = \arg \underset{i \le i \le N}{\max}[\gamma_t(i)]\tag{32}
$$

### Viterbi算法

Viterbi算法使用动态规划的思想，降低了求解问题的时间复杂度。其算法流程如下所示:

>Input: $\lambda = (A, B, \pi)$和观测$O = (o_1, o_2, \ldots, o_T)$
>
>Output: 最优路径$I^* = (i_1^*, i_2^*, \ldots, i_T^*)$
>
>(1)初始化
>$$
>\delta_1(i)  = \pi_ib_i(o_1), \ \ \ i = 1,2 ,\ldots, N\\\varPsi_1(i) = 0,\ \ \ i = 1, 2, \ldots, N
>$$
>(2)递推， 对$t = 2, 3, \ldots, T$
>$$
>\delta_t(i) = \underset{1\le j \le N}{\max}\big[\delta_{t-1}(j)a_{ji}\big]b_i(o_t), \ \ \ i =1 ,2 , \ldots, N\\\varPsi_t(i) = \arg\underset{1\le j\le N}{\max}\big[\delta_{t-1}(j)a_{ji}\big], \ \ \ i = 1,2 ,\ldots,N
>$$
>(3)终止
>$$
>P^* = \underset{1\le i\le N}{\max}\delta_T(i)\\i_T^* = \arg\underset{1\le i\le N}{\max}\big[\delta_T(i)\big]
>$$
>(4)溯回，对$t = T-1, T-2, \ldots, 1$
>$$
>i_t^* = \varPsi_{t+1}(i_{t+1}^*)
>$$